# Multiple Dispatch

Every case of defining function we saw earlier works mostly the same as in Python. Where Julia really shines is in extending this system by answering the two questions we posed at the end of that section:

- How do we let functions handle multiple combinations of input?
- How do we extend functionality we don't own?

We already saw how to specialize functions to certain types of inputs:

In [1]:
function diagsum{T<:Number}(A::Array{T, 2})  # all matrices with elements that are numbers
    return sum(diag(A))
end

diagsum (generic function with 1 method)

In [2]:
A = rand(5, 5)
diagsum(A)

3.3132089407799397

But we might also want to handle the case of a vector: 

In [4]:
diagsum(rand(7))

LoadError: LoadError: ArgumentError: use diagm instead of diag to construct a diagonal matrix
while loading In[4], in expression starting on line 1

For classes, we'd attach this behavior to the object `vector.diagsum` where it made sense, but this can be problematic:
- what if we don't own `vector`? Then we need to subclass or compose.
- what if the function takes multiple arguments (like `*`). Who should own it? What if we had a custom sparse array type and wanted to define its arithmetic interactions with preexisting types?
- what class should "own" an optimization algorithm?
- what if we want to special-case certain types of inputs, but it's not our code?

This way of separating data structures and algorithms is particularly useful in numerical computing:

In Julia, behaviors aren't *owned*, they're *special-cased*:

In [5]:
# Vector{T} is an alias for Array{T, 1}
diagsum{T<:Number}(v::Vector{T}) = sum(v)
diagsum(x::Number) = x

diagsum (generic function with 3 methods)

Note, three methods! The arguments determine which gets called:

In [6]:
@which diagsum(5)

diagsum(x::Number) at In[5]:3

In [7]:
@which diagsum(rand(4))

diagsum{T<:Number}(v::Array{T<:Number,1}) at In[5]:2

Note how much boilerplate type checking and special casing we avoid!

In [8]:
methods(+)

# 171 methods for generic function "+":
+(x::Bool) at bool.jl:33
+(x::Bool, y::Bool) at bool.jl:36
+(y::AbstractFloat, x::Bool) at bool.jl:46
+(x::Int64, y::Int64) at int.jl:8
+(x::Int8, y::Int8) at int.jl:16
+(x::UInt8, y::UInt8) at int.jl:16
+(x::Int16, y::Int16) at int.jl:16
+(x::UInt16, y::UInt16) at int.jl:16
+(x::Int32, y::Int32) at int.jl:16
+(x::UInt32, y::UInt32) at int.jl:16
+(x::UInt64, y::UInt64) at int.jl:16
+(x::Int128, y::Int128) at int.jl:16
+(x::UInt128, y::UInt128) at int.jl:16
+(x::Integer, y::Ptr{T}) at pointer.jl:77
+(x::Float32, y::Float32) at float.jl:207
+(x::Float64, y::Float64) at float.jl:208
+(z::Complex{T<:Real}, w::Complex{T<:Real}) at complex.jl:111
+(x::Bool, z::Complex{Bool}) at complex.jl:118
+(z::Complex{Bool}, x::Bool) at complex.jl:119
+(x::Bool, z::Complex{T<:Real}) at complex.jl:125
+(z::Complex{T<:Real}, x::Bool) at complex.jl:126
+(x::Real, z::Complex{Bool}) at complex.jl:132
+(z::Complex{Bool}, x::Real) at complex.jl:133
+(x::Real, z::Complex{T<:Real}) at complex.jl:144
+(z::Complex{T<:Real}, x::Real) at complex.jl:145
+(x::Rational{T<:Integer}, y::Rational{T<:Integer}) at rational.jl:179
+(x::Bool, A::AbstractArray{Bool,N}) at arraymath.jl:136
+(x::Integer, y::Char) at char.jl:43
+(a::Float16, b::Float16) at float16.jl:136
+(x::BigInt, y::BigInt) at gmp.jl:256
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:279
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:285
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:292
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:304
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:308
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:320
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:321
+(x::BigFloat, y::BigFloat) at mpfr.jl:208
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:215
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:219
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:223
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:227
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:231
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:235
+(x::BigFloat, c::BigInt) at mpfr.jl:239
+(c::BigInt, x::BigFloat) at mpfr.jl:243
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:379
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:385
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:392
+(x::Irrational{sym}, y::Irrational{sym}) at irrationals.jl:72
+(x::Number) at operators.jl:73
+{T<:Number}(x::T<:Number, y::T<:Number) at promotion.jl:211
+{T<:AbstractFloat}(x::Bool, y::T<:AbstractFloat) at bool.jl:43
+(x::Number, y::Number) at promotion.jl:167
+(r1::OrdinalRange{T,S}, r2::OrdinalRange{T,S}) at operators.jl:330
+{T<:AbstractFloat}(r1::FloatRange{T<:AbstractFloat}, r2::FloatRange{T<:AbstractFloat}) at operators.jl:337
+{T<:AbstractFloat}(r1::LinSpace{T<:AbstractFloat}, r2::LinSpace{T<:AbstractFloat}) at operators.jl:356
+(r1::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}, r2::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}) at operators.jl:369
+(x::Ptr{T}, y::Integer) at pointer.jl:75
+{S,T}(A::Range{S}, B::Range{T}) at arraymath.jl:69
+{S,T}(A::Range{S}, B::AbstractArray{T,N}) at arraymath.jl:87
+(A::BitArray{N}, B::BitArray{N}) at bitarray.jl:859
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:28
+(A::Array{T,2}, B::Diagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Bidiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Tridiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::SymTridiagonal{T}) at linalg/special.jl:131
+(A::Array{T,2}, B::Base.LinAlg.AbstractTriangular{T,S<:AbstractArray{T,2}}) at linalg/special.jl:159
+(A::Array{T,N}, B::SparseMatrixCSC{Tv,Ti<:Integer}) at sparse/sparsematrix.jl:1019
+{P<:Union{Base.Dates.CompoundPeriod,Base.Dates.Period}}(x::Union{De

# Caution:

New Julia users frequently go around peppering function declarations with type restrictions. This is often unnecessary. 

Consider:

In [9]:
doubler(x) = 2x

doubler (generic function with 1 method)

Shouldn't you have

```julia
doubler(x::Float64)
```
for performance?

**No**. The JIT compiles this function once for every *concrete* type you call it with. So 
`doubler(1)` and `doubler(1.5)` each get compiled separately, and Julia selects based on the type of the input which one to use thereafter.

So...

**Use type restrictions in functions only to control what code gets executed. Type as loosely as possible.**

### Another caveat:

When we declare:

```julia
doubler(x=0) = 2x
```
Julia actually defines *two* methods, `doubler(x)` and `doubler()`. In the second case, it just makes the substitution. That is, default arguments are just sugar for multiple dispatch. Note, though, that a lot of default arguments does mean a lot of extra methods. If you have a lot of optional arguments, you may want keyword arguments, for which only a single method is compiled.

# Q: How does this get used?

## A: *Everywhere*

- special-case algorithms for sparse vs dense matrices, diagonal matrices, matrices vs vectors
- constructors (as we'll see): create output from multiple types of inputs
- for parsing code (macros): define start condition, end conditions, recurse